In [2]:
# 🧩 Causal LM SFT with LoRA on KoGPT2
!pip -q install -U transformers datasets peft accelerate sentencepiece

import os, random, numpy as np, torch
from dataclasses import dataclass
from typing import Dict, List
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

# ========== 0) Repro & perf ==========
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cuda.matmul.allow_tf32 = True

# ========== 1) Tokenizer / Model ==========
BASE_MODEL = "skt/kogpt2-base-v2"
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

# 템플릿용 특수 토큰(토큰 경계 안정화)
B_INST = "### Instruction:"
B_RESP = "### Response:"
SPECIAL_TOKENS = {"additional_special_tokens": [B_INST, B_RESP]}
tok.add_special_tokens(SPECIAL_TOKENS)

if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
model.resize_token_embeddings(len(tok))

# ========== 2) LoRA 설정 ==========
peft_conf = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none",
    task_type=TaskType.CAUSAL_LM,
    # GPT-2 계열 호환 타깃 모듈
    target_modules=["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"]
)
model = get_peft_model(model, peft_conf)
model.print_trainable_parameters()

# ========== 3) 소형 지시문 데이터셋 ==========
pairs = [
    {"prompt":"날씨 요약 규칙: 1) 한 줄 2) 이모지 금지\n서울 오늘 날씨 알려줘.",
     "response":"서울은 맑고 낮기온 28도, 미세먼지 보통입니다."},
    {"prompt":"한 줄로 요약: '대중교통 요금 인상 논의가 진행 중이다.'",
     "response":"대중교통 요금 인상이 논의 단계에 있다."},
    {"prompt":"간단 번역: '사과는 건강에 좋다' -> 영어",
     "response":"Apples are good for health."},
    {"prompt":"비즈니스 이메일 첫 문장 제안(한국어, 공손체): 납기 연장 요청",
     "response":"안녕하세요, 귀사 프로젝트의 납기 일정 관련하여 조심스럽게 연장을 요청드립니다."},
]

def format_example(p, r, eos):
    return f"{B_INST}\n{p}\n\n{B_RESP}\n{r}{eos}"

train_texts = [format_example(d["prompt"], d["response"], tok.eos_token) for d in pairs]

# 소량 데이터 → upsampling으로 수렴 안정화
REPEAT = 60  # 필요 시 30~200에서 조절
train_texts = train_texts * REPEAT

raw_ds = Dataset.from_dict({"text": train_texts})

# ========== 4) 토크나이즈 + 레이블 마스킹(응답만 loss) ==========
def build_features(batch):
    texts = batch["text"]
    input_ids_list, attn_list, labels_list = [], [], []
    # "### Response:\n" 토큰 시퀀스
    resp_tag_ids = tok(B_RESP + "\n", add_special_tokens=False)["input_ids"]

    def find_subseq(seq, sub):
        L, l = len(seq), len(sub)
        for i in range(L - l + 1):
            if seq[i:i+l] == sub:
                return i
        return -1

    for t in texts:
        enc = tok(t, max_length=512, truncation=True)
        input_ids = enc["input_ids"]
        attn = enc["attention_mask"]

        idx = find_subseq(input_ids, resp_tag_ids)
        if idx == -1:
            # 안전장치: 태그를 못 찾으면 전체 -100
            labels = [-100] * len(input_ids)
        else:
            start = idx + len(resp_tag_ids)
            labels = [-100] * start + input_ids[start:]

        input_ids_list.append(input_ids)
        attn_list.append(attn)
        labels_list.append(labels)

    return {"input_ids": input_ids_list, "attention_mask": attn_list, "labels": labels_list}

ds_tok = raw_ds.map(build_features, batched=True, remove_columns=["text"])

# ========== 5) Collator: labels는 수동 패딩 ==========
@dataclass
class ResponseOnlyCollator:
    tokenizer: AutoTokenizer
    pad_to_multiple_of: int = 8

    def __call__(self, features: List[Dict]):
        # 1) labels를 잠시 분리해 tokenizer.pad가 건드리지 않게 함
        labels_list = [f.pop("labels") for f in features]

        # 2) 입력만 패딩
        batch = self.tokenizer.pad(
            features,
            padding=True,
            max_length=None,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt"
        )

        # 3) labels 수동 패딩(-100) 후 텐서화
        max_len = batch["input_ids"].size(1)
        padded_labels = []
        for lab in labels_list:
            if len(lab) < max_len:
                lab = lab + [-100] * (max_len - len(lab))
            else:
                lab = lab[:max_len]
            padded_labels.append(lab)
        batch["labels"] = torch.tensor(padded_labels, dtype=torch.long)
        return batch

collator = ResponseOnlyCollator(tok)

# ========== 6) 학습 세팅 ==========
try:
    bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
except Exception:
    bf16_ok = False
fp16_ok = torch.cuda.is_available() and not bf16_ok

args = TrainingArguments(
    output_dir="./kogpt2-lora-sft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,          # 소량 데이터 → 낮게
    num_train_epochs=8,          # 필요 시 6~20 사이에서 조절
    lr_scheduler_type="cosine",
    weight_decay=0.0,
    logging_steps=10,
    save_strategy="no",
    bf16=bf16_ok,
    fp16=fp16_ok,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok,
    tokenizer=tok,
    data_collator=collator
)

trainer.train()

# ========== 7) 추론 유틸 (학습 템플릿과 동일) ==========
def generate(prompt, max_new_tokens=80, do_sample=False, top_p=0.9, temperature=0.7):
    text = f"{B_INST}\n{prompt}\n\n{B_RESP}\n"
    inputs = tok(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,    # 소량 데이터 과적합 → 기본 False로 보수적 생성
            top_p=top_p, temperature=temperature,
            pad_token_id=tok.eos_token_id,
            eos_token_id=tok.eos_token_id
        )
    full = tok.decode(out[0], skip_special_tokens=False)
    # 응답 부분만 추출
    if B_RESP in full:
        ans = full.split(B_RESP, 1)[-1].strip()
    else:
        ans = full
    return ans.strip()

print("=== 데모 출력 ===")
tests = [
    "서울 내일 날씨를 한 줄로 요약해줘.",
    "정중한 일정 조율 메일 첫 문장 써줘.",
    "한 줄로 요약: '도로 확장 공사가 지연되고 있다.'",
    "간단 번역: '포도는 항산화 효과가 있다' -> 영어",
]
for p in tests:
    print(p, "->", generate(p))


trainable params: 1,179,648 || all params: 126,345,984 || trainable%: 0.9337


Map:   0%|          | 0/240 [00:00<?, ? examples/s]

/tmp/ipython-input-173637014.py:155: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 51200, 'bos_token_id': 51200, 'pad_token_id': 51200}.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.413000
20,3.181700
30,2.125500
40,1.423000
50,1.092000
60,0.942600
70,0.850200
80,0.861500
90,0.843900
100,0.833900


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== 데모 출력 ===
서울 내일 날씨를 한 줄로 요약해줘. -> 서울은 맑고 낮기온 28도, 미세먼지 보통입니다. ^^ 』고 말합니다. ^^ 。서울은 맑고 낮기온 28도, 미세먼지 보통입니다. ^^ 。서울은 맑고 낮기온 28도, 미세먼지 보통입니다. ^^ 。서울은 맑고 낮기온 28도, 미세먼지 보통입니다. ^^ 。서울은 맑고 낮기온 28도, 미세먼
정중한 일정 조율 메일 첫 문장 써줘. -> 안녕하세요, 귀사 프로젝트의 성공을 거두었습니다. health.presented.go.kr/spectes are good for health.go.kr/spectes.go.kr/spare good for health.go.kr)에서 확인하세요. health.go.kr/spectes.go.kr
한 줄로 요약: '도로 확장 공사가 지연되고 있다.' -> 도로 확장 공사가 지연되었다."
도로 확장 공사가 지연되고 있다. health. health. health. health. health. health. health. health. health. health. health. health. health. heal
간단 번역: '포도는 항산화 효과가 있다' -> 영어 -> Apples are good for health. health. health. health. health. health. health. are good for health. health. health. health. health. health.
